In [1]:
%matplotlib notebook
#Import library to read from Twitter API
import tweepy

In [ ]:
#Step1 Processing the data

In [4]:
#Step2 Create a Neural Network
#Import all libraries for creating a deep neural network
import keras
#Sequential is the standard type of neural network with stackable layers
from keras.models import Sequential
#Dense: Standard layers with every node connected, dropout: avoids overfitting
from keras.layers import Dense, Dropout, Activation

In [5]:
#Create the model
model_nn = Sequential()